<a href="https://colab.research.google.com/github/ydesquitado/APM110_SA2/blob/main/SA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

path = "/content/drive/MyDrive/BTC Data with Datetime.csv"

In [4]:
df = pd.read_csv(path)
filt = df['Datetime'] >= '2012-01-01'

dff = df[filt]

dff = dff.dropna()
dff.fillna(0)

,Datetime,UNIX Timestamp (ms),Open,High,Low,Close,volume,Volume_(Currency),Weighted_Price
1224,2012-01-01 04:16:00,1325391360000,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
1896,2012-01-01 15:28:00,1325431680000,4.84,4.84,4.84,4.84,10.000000,48.400000,4.840000
2333,2012-01-01 22:45:00,1325457900000,5.00,5.00,5.00,5.00,10.100000,50.500000,5.000000
3612,2012-01-02 20:04:00,1325534640000,5.00,5.00,5.00,5.00,19.048000,95.240000,5.000000
4553,2012-01-03 11:45:00,1325591100000,5.32,5.32,5.32,5.32,2.419173,12.870000,5.320000
...,...,...,...,...,...,...,...,...,...
4857372,2021-03-30 23:56:00,1617148560000,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,2021-03-30 23:57:00,1617148620000,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,2021-03-30 23:58:00,1617148680000,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,2021-03-30 23:59:00,1617148740000,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [5]:
to_returns = dff['Close'].pct_change()
dff['Returns'] = to_returns

In [6]:
# dff.drop(['Return'], axis = 1, inplace = True)
# dff.drop(['Mid'], axis = 1, inplace = True)

In [16]:
high_low_ave = ['High', 'Low']
dff['Mid'] = dff[high_low_ave].mean(axis=1)

returns = dff['Returns']
returns = returns.dropna()

1896       0.056769
2333       0.033058
3612       0.000000
4553       0.064000
4710      -0.033835
             ...   
4857372   -0.000482
4857373   -0.000003
4857374    0.000648
4857375    0.000626
4857376    0.000299
Name: Returns, Length: 3613764, dtype: float64

In [ ]:
plt.hist(dff['Returns'], bins='auto', density=True, color='green', edgecolor='black')

plt.xlabel('Returns')
plt.ylabel('Density')
plt.title('Histogram of Bitcoin Returns')

plt.show()

# Normal Distribution

In [23]:
params_norm = stats.norm.fit(returns)
params_norm

(0.00011281664590589813, 0.20431613263762724)

# Laplace Distribution

In [20]:
params_laplace = stats.laplace.fit(returns)
params_laplace

(0.0, 0.0011753500091878123)

# Gamma Distribution

In [21]:
params_gamma = stats.gamma.fit(returns)
params_gamma

(1.838410389244527e-06, -0.13286713286713295, 0.6913967769688074)

# Exponential Distribution

In [22]:
params_expon = stats.expon.fit(returns)
params_expon

(-0.9974514068234335, 0.9975642234693394)

In [38]:
ks_norm = stats.kstest(returns, stats.norm.cdf, args = params_norm)

ks_laplace = stats.kstest(returns, stats.laplace.cdf, args = params_laplace)

ks_gamma = stats.kstest(returns, stats.gamma.cdf, args = params_gamma)

ks_expon = stats.kstest(returns, stats.expon.cdf, args = params_expon)

In [39]:
print(ks_norm)
print(ks_laplace)
print(ks_gamma)
print(ks_expon)

KstestResult(statistic=0.47944936920479986, pvalue=0.0, statistic_location=0.007670672462285921, statistic_sign=1)
KstestResult(statistic=0.09734984275462344, pvalue=0.0, statistic_location=-0.00013532003187544106, statistic_sign=-1)
KstestResult(statistic=0.999987955841104, pvalue=0.0, statistic_location=-0.12142857142857133, statistic_sign=-1)
KstestResult(statistic=0.6256241456833058, pvalue=0.0, statistic_location=-0.012500000000000067, statistic_sign=-1)


# Conclusion

According to the [article](https://medium.com/analytics-vidhya/modelling-bitcoin-returns-e6c3d286c00f) by Pedro Guilherme Frade Moro, the lower the D, a statistic obtained from the Kolmogorov-Smirnov Test (kstest), "*indicates that the probability that the samples correspond to the same distribution is greater.*" Among the bunch, Normal, Laplace, Gamma, and Exponential, Laplace has the lowest D value, followed by Normal, then Exponential, and lastly Gamma.

Therefore, it is safe to conclude that **Laplace Distribution** is the best among the tested distributions.